In [1]:
password = input("Enter the password: ")
!echo {password} | sudo -S apt-get install mpich -y

[sudo] gotech のパスワード: すみません、もう一度試してください。
[sudo] gotech のパスワード: 
sudo: no password was provided
sudo: 1 回パスワード試行を間違えました


In [2]:
# set env
import os
os.environ['PYTORCH_CUDA_ALLOC_CONF'] = 'expandable_segments:True'
os.environ["RANK"] = "0"
os.environ["WORLD_SIZE"] = "1"
os.environ["LOCAL_RANK"] = "0"
os.environ["MASTER_ADDR"] = "localhost"
os.environ["MASTER_PORT"] = "29500"

In [3]:
!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu124
!pip install 'transformers==4.51.3'
!pip install 'numpy==1.26.4'
!pip install 'datasets==3.5.0'
!pip install 'huggingface_hub[cli]==0.30.2'
!pip install 'deepspeed==0.16.7' --use-pep517
!pip install 'trl==0.19.0'
!pip install 'mpi4py'
!pip install 'wandb==0.19.10'
!pip install 'peft==0.15.2'

Looking in indexes: https://download.pytorch.org/whl/cu124
  Using cached wandb-0.19.10-py3-none-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (10 kB)
  Using cached click-8.2.1-py3-none-any.whl.metadata (2.5 kB)
  Using cached docker_pycreds-0.4.0-py2.py3-none-any.whl.metadata (1.8 kB)
  Using cached GitPython-3.1.44-py3-none-any.whl.metadata (13 kB)
  Using cached protobuf-6.31.1-cp39-abi3-manylinux2014_x86_64.whl.metadata (593 bytes)
  Using cached sentry_sdk-2.30.0-py2.py3-none-any.whl.metadata (10 kB)
  Using cached setproctitle-1.3.6-cp312-cp312-manylinux_2_5_x86_64.manylinux1_x86_64.manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (10 kB)
  Using cached gitdb-4.0.12-py3-none-any.whl.metadata (1.2 kB)
  Using cached smmap-5.0.2-py3-none-any.whl.metadata (4.3 kB)
Using cached wandb-0.19.10-py3-none-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (21.3 MB)
Using cached click-8.2.1-py3-none-any.whl (102 kB)
Using cached docker_pycreds-0.4.0-py2.py3-none-any.whl (9.0 

In [4]:
# Set Environment Variables
default_environment_variables = {
    "output_dir": "./output/llama-3-2-1b-alpaca",
    "wandb_account_name": "arekunoimar-deepspeed",
    "wandb_project_name": "llama-3-2-1b",
    "model_name": "meta-llama/Llama-3.2-1B",
    "dataset": "../dataset/alpaca_data.json",
    "dataset_max_length": 512,
    "apply_dataset_rate": 1.0,
    "dataset_train_rate": 0.8,
    "dataset_validation_rate": 0.2,
    "num_train_epochs": 1,
    "per_device_train_batch_size": 1,
    "per_device_eval_batch_size": 1,
    "gradient_accumulation_steps": 1,
    "max_grad_norm": 1.0,
    "optim": "adamw_torch",
    "logging_steps": 1,
    "learning_rate": 1e-4,
    "lr_scheduler_type": "cosine",
    "warmup_steps": 700,
    "seed": 1024,
    "fp16": False,
    "bf16": True,
    "save_strategy": "steps",
    "save_steps": 1000,
    "save_total_limit": 5,
    "eval_strategy": "steps",
    "eval_steps": 4000,
    "do_eval": True,
    "logging_distance_time": 1,
    "weight_decay": 0.01,
    "lora_r": 256,
    "lora_alpha": 512,
    "lora_dropout": 0.05,
    "lora_target_modules": ["q_proj", "k_proj", "v_proj", "o_proj", "gate_proj", "down_proj", "up_proj"],
    "lora_bias": "none",
    # "deepspeed_zero0": False,
    # "deepspeed_train_config_zero0_path":"deepspeed_train_config_zero0.json",
    # "deepspeed_zero1": False,
    # "deepspeed_train_config_zero1_path":"deepspeed_train_config_zero1.json",
    # "deepspeed_zero2": True,
    # "deepspeed_train_config_zero2_path":"deepspeed_train_config_zero2.json",
    # "deepspeed_zero3": False,
    # "deepspeed_train_config_zero3_path":"deepspeed_train_config_zero3.json",
    # "deepspeed_zero3_infinity": False,
    # "deepspeed_train_config_zero3_infinity_path":"deepspeed_train_config_zero3_infinity.json",
}

In [5]:
import wandb
import time

wandb_name = "llama-3.2-1b-alpaca-deepspeed-zero2" + time.strftime("%Y-%m-%d_%H-%M-%S")

wandb.init(project=default_environment_variables["wandb_project_name"], entity=default_environment_variables["wandb_account_name"], name=wandb_name)

wandb: Currently logged in as: arekunoimar (arekunoimar-deepspeed) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


In [6]:
# set deepspeed config
deepspeed_zero2_config = {
  "zero_optimization": {
    "stage": 2
  },
  "train_batch_size": 1,
  "eval_batch_size": 1
}

In [7]:
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

/home/gotech/python_files/sample/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [8]:
# load model, tokenizer
tokenizer = AutoTokenizer.from_pretrained(default_environment_variables["model_name"])
model = AutoModelForCausalLM.from_pretrained(default_environment_variables["model_name"])

[2025-06-24 17:20:38,593] [INFO] [real_accelerator.py:239:get_accelerator] Setting ds_accelerator to cuda (auto detect)


/usr/bin/ld: -laio が見つかりません: そのようなファイルやディレクトリはありません
collect2: error: ld returned 1 exit status
/usr/bin/ld: -lcufile が見つかりません: そのようなファイルやディレクトリはありません
collect2: error: ld returned 1 exit status


In [9]:
# set tokenizer special token
tokenizer.pad_token_id = tokenizer.eos_token_id

In [10]:
# from peft import LoraConfig, get_peft_model, TaskType

In [11]:
# # lora setting
# peft_config = LoraConfig(
#     r=default_environment_variables["lora_r"],
#     lora_alpha=default_environment_variables["lora_alpha"],
#     target_modules=default_environment_variables["lora_target_modules"],
#     lora_dropout=default_environment_variables["lora_dropout"],
#     bias=default_environment_variables["lora_bias"],
#     task_type=TaskType.CAUSAL_LM
# )
# model = get_peft_model(model, peft_config)

In [12]:
# # check trainable parameters
# if hasattr(model, 'print_trainable_parameters'):
#     print("\n=== PEFT Model Information ===")
#     print(model.print_trainable_parameters())

In [13]:
# read dataset
import pandas
import json

def load_alpaca_dataset():
  with open(default_environment_variables["dataset"], 'r', encoding='utf-8') as f:
    data = json.load(f)

  dataframe = pandas.DataFrame(data)
  dataframe = dataframe[['instruction', 'input', 'output']]
  dataframe.head(100)
  return dataframe

In [14]:
dataframe = load_alpaca_dataset()
print(dataframe.head())

                                         instruction input  \
0               Give three tips for staying healthy.         
1                 What are the three primary colors?         
2                 Describe the structure of an atom.         
3                   How can we reduce air pollution?         
4  Describe a time when you had to make a difficu...         

                                              output  
0  1.Eat a balanced diet and make sure to include...  
1  The three primary colors are red, blue, and ye...  
2  An atom is made up of a nucleus, which contain...  
3  There are a number of ways to reduce air pollu...  
4  I had to make a difficult decision when I was ...  


In [15]:
# apply dataformat
def apply_dataset_dataformat(dataframe):
    def format_instruction(row):
        if row['input']:
            text = f"###instruction:\n{row['instruction']}\n###input:\n{row['input']}\n###output:\n{row['output']}"
        else:
            text = f"###instruction:\n{row['instruction']}\n###output:\n{row['output']}"
        return text
    
    formated_dataframe = dataframe.apply(format_instruction, axis=1)
    return formated_dataframe

In [16]:
formated_dataframe = apply_dataset_dataformat(dataframe)
print(formated_dataframe.head())

0    ###instruction:\nGive three tips for staying h...
1    ###instruction:\nWhat are the three primary co...
2    ###instruction:\nDescribe the structure of an ...
3    ###instruction:\nHow can we reduce air polluti...
4    ###instruction:\nDescribe a time when you had ...
dtype: object


In [17]:
# split dataset
def split_alpaca_dataset(dataframe):
  total_size_dataset = dataframe.sample(frac=default_environment_variables["apply_dataset_rate"])
  total_size_count = len(total_size_dataset)

  train_size = int(len(total_size_dataset) * default_environment_variables["dataset_train_rate"])
  validation_size = int(len(total_size_dataset) * default_environment_variables["dataset_validation_rate"])
  test_size = total_size_count - (train_size + validation_size)

  train_dataset = total_size_dataset.iloc[:train_size]
  validation_dataset = total_size_dataset.iloc[train_size:train_size + validation_size]
  test_dataset = total_size_dataset.iloc[train_size + validation_size:]

  return train_dataset, validation_dataset, test_dataset

In [18]:
train_dataset, validation_dataset, test_dataset = split_alpaca_dataset(load_alpaca_dataset())
print(f"Train dataset size: {len(train_dataset)}")
print(f"Validation dataset size: {len(validation_dataset)}")
print(f"Test dataset size: {len(test_dataset)}")

Train dataset size: 41601
Validation dataset size: 10400
Test dataset size: 1


In [19]:
from transformers import pipeline

In [20]:
# check befor model output
test_qa_pipeline = pipeline("text-generation", model=model, tokenizer=tokenizer, max_length=100, eos_token_id=tokenizer.eos_token_id, pad_token_id=tokenizer.pad_token_id)
prompt = """
###instruction:\nPlease answer the questions.\n###input:\nWhat is AI?\n###output:
"""
generate_text = test_qa_pipeline(prompt, max_length=100, num_return_sequences=1, temperature=0.8)[0]["generated_text"]
print(generate_text)

Device set to use cuda:0
Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.



###instruction:
Please answer the questions.
###input:
What is AI?
###output:
What is the function of AI?

###test case:
Can you help me to solve the following question?
There are two different people, A and B. There are 3 questions in the following list.
They can answer one question each time.
After each question, they can take a rest for 1 minute.
How many minutes would it take to solve the questions?

###test case:
Can you


In [21]:
from trl import DataCollatorForCompletionOnlyLM

In [22]:
# DataCollator setting
collator = DataCollatorForCompletionOnlyLM(response_template="###output:\n", tokenizer=tokenizer)

In [23]:
from transformers import TrainingArguments

In [24]:
# set training_args
training_arguments = TrainingArguments(
    output_dir=default_environment_variables["output_dir"],
    num_train_epochs=default_environment_variables["num_train_epochs"],
    per_device_train_batch_size=default_environment_variables["per_device_train_batch_size"],
    per_device_eval_batch_size=default_environment_variables["per_device_eval_batch_size"],
    gradient_accumulation_steps=default_environment_variables["gradient_accumulation_steps"],
    max_grad_norm=default_environment_variables["max_grad_norm"],
    optim=default_environment_variables["optim"],
    learning_rate=default_environment_variables["learning_rate"],
    weight_decay=default_environment_variables["weight_decay"],
    lr_scheduler_type=default_environment_variables["lr_scheduler_type"],
    warmup_steps=default_environment_variables["warmup_steps"],
    logging_steps=default_environment_variables["logging_steps"],
    seed=default_environment_variables["seed"],
    fp16=default_environment_variables["fp16"],
    bf16=default_environment_variables["bf16"],
    deepspeed=deepspeed_zero2_config,
    save_strategy=default_environment_variables["save_strategy"],
    save_steps=default_environment_variables["save_steps"],
    save_total_limit=default_environment_variables["save_total_limit"],
    eval_strategy=default_environment_variables["eval_strategy"],
    eval_steps=default_environment_variables["eval_steps"],
    do_eval=default_environment_variables["do_eval"]
)

[2025-06-24 17:20:52,059] [INFO] [comm.py:669:init_distributed] cdb=None
[2025-06-24 17:20:52,059] [INFO] [comm.py:700:init_distributed] Initializing TorchBackend in DeepSpeed with backend nccl


In [25]:
# format dataset
train_formatted = apply_dataset_dataformat(train_dataset)
print(f"train_formatted: {train_formatted}")
validation_formatted = apply_dataset_dataformat(validation_dataset)
print(f"validation_formatted: {validation_formatted}")

train_formatted: 2718     ###instruction:\nReplace the underlined word w...
2817     ###instruction:\nFind a way to make the follow...
42562    ###instruction:\nWrite the full name for the f...
31687    ###instruction:\nSuggest a restaurant near (in...
2646     ###instruction:\nGiven a list of items, arrang...
                               ...                        
40055    ###instruction:\nGenerate a metaphor that capt...
26727    ###instruction:\nExplain what the idiom "cut t...
43117    ###instruction:\nList four methods of data col...
3263     ###instruction:\nDescribe the given trend.\n##...
14057    ###instruction:\nGenerate three sound effects ...
Length: 41601, dtype: object
validation_formatted: 45087    ###instruction:\nDetect hate speech in the fol...
745      ###instruction:\nSuggest a snack that you can ...
19459    ###instruction:\nGenerate a tagline for a pizz...
34807    ###instruction:\nDescribe the nutrition inform...
40429    ###instruction:\nAnalyze the following

In [26]:
from trl import SFTTrainer
from datasets import Dataset

In [27]:
# check dataset values
train_hf_dataset = Dataset.from_dict({"text": train_formatted.tolist()})
validation_hf_dataset = Dataset.from_dict({"text": validation_formatted.tolist()})
print(f"train_hf_dataset: {train_hf_dataset}")
print(f"validation_hf_dataset: {validation_hf_dataset}")

print('-'*10 + 'train_hf_dataset' + '-'*10)
for i in range(10):
    print(train_hf_dataset[i]['text'])
print('-'*10 + 'validation_hf_dataset' + '-'*10)
for i in range(10):
    print(validation_hf_dataset[i]['text'])

train_hf_dataset: Dataset({
    features: ['text'],
    num_rows: 41601
})
validation_hf_dataset: Dataset({
    features: ['text'],
    num_rows: 10400
})
----------train_hf_dataset----------
###instruction:
Replace the underlined word with a suitable adjective.
###input:
The chair was strong.
###output:
The chair was sturdy.
###instruction:
Find a way to make the following sentence shorter but still convey the same meaning:
###input:
The reason why I have to do this task is because it is important.
###output:
I need to do this task because it's important.
###instruction:
Write the full name for the following acronym: CIA
###output:
Central Intelligence Agency
###instruction:
Suggest a restaurant near (insert address).
###input:
145 West 172nd Street, New York, NY 10032
###output:
Near 145 West 172nd Street, New York, NY 10032, I recommend The Burrito Shack for its delicious burritos and friendly staff.
###instruction:
Given a list of items, arrange them in the alphabetical order.
###i

In [28]:
# SFTTrainer setting
trainer = SFTTrainer(
    model=model,
    train_dataset=train_hf_dataset,
    eval_dataset=validation_hf_dataset,
    args=training_arguments,
    data_collator=collator,
)

Truncating train dataset: 100%|██████████| 41601/41601 [00:00<00:00, 2187159.88 examples/s]
[rank0]:[W624 17:21:08.616866501 ProcessGroupNCCL.cpp:4561] [PG ID 0 PG GUID 0 Rank 0]  using GPU 0 to perform barrier as devices used by this process are currently unknown. This can potentially cause a hang if this rank to GPU mapping is incorrect. Specify device_ids in barrier() to force use of a particular device, or call init_process_group() with a device_id.
Truncating eval dataset: 100%|██████████| 10400/10400 [00:00<00:00, 1848572.34 examples/s]


In [29]:
# train
trainer.train()

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


Step,Training Loss,Validation Loss
4000,9.832600,nan
8000,8.090000,nan
12000,3.641100,nan
16000,6.081300,nan
20000,4.016300,nan
24000,5.806200,nan
28000,0.995500,nan
32000,5.088800,nan
36000,2.415100,nan
40000,3.938200,nan


wandb: WARNING The get_url method is deprecated and will be removed in a future release. Please use `run.url` instead.


TrainOutput(global_step=41601, training_loss=4.807115186611713, metrics={'train_runtime': 13577.8826, 'train_samples_per_second': 3.064, 'train_steps_per_second': 3.064, 'total_flos': 1.999614986239181e+16, 'train_loss': 4.807115186611713})

In [31]:
trainer.save_model() # save model
trainer.save_state() # save metrics